<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-data" data-toc-modified-id="Import-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import data</a></span></li><li><span><a href="#Q1" data-toc-modified-id="Q1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Q1</a></span></li><li><span><a href="#Q2" data-toc-modified-id="Q2-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Q2</a></span></li><li><span><a href="#Q3" data-toc-modified-id="Q3-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Q3</a></span></li><li><span><a href="#Q4" data-toc-modified-id="Q4-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Q4</a></span><ul class="toc-item"><li><span><a href="#Include-users-with-only-1-record" data-toc-modified-id="Include-users-with-only-1-record-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Include users with only 1 record</a></span></li><li><span><a href="#Ignore-users-with-only-1-record" data-toc-modified-id="Ignore-users-with-only-1-record-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Ignore users with only 1 record</a></span></li></ul></li></ul></div>

* Author: Stephanie Jung

# Import data

In [4]:
import pandas as pd
from sqlalchemy import create_engine, text
pd.options.display.max_rows=10

In [5]:
engine = create_engine('sqlite://')
df = pd.read_csv('q1_data.csv', parse_dates=['ts']).to_sql('db', engine, if_exists='replace', index=False)

In [6]:
def runQuery(sql):
    result = engine.connect().execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())

In [7]:
q = """SELECT *
FROM db
"""
runQuery(q)

,ts,user_id,country_id,site_id
0,2019-02-01 00:01:24.000000,LC36FC,TL6,N0OTG
1,2019-02-01 00:10:19.000000,LC39B6,TL6,N0OTG
2,2019-02-01 00:21:50.000000,LC3500,TL6,N0OTG
3,2019-02-01 00:22:50.000000,LC374F,TL6,N0OTG
4,2019-02-01 00:23:44.000000,LCC1C3,TL6,QGO3G
...,...,...,...,...
3548,2019-02-07 23:56:57.000000,LC3F13,TL6,QGO3G
3549,2019-02-07 23:58:36.000000,LC3842,HVQ,3POLC
3550,2019-02-07 23:58:56.000000,LC35EB,TL6,QGO3G
3551,2019-02-07 23:59:19.000000,LC3842,HVQ,3POLC


# Q1

Consider only the rows with country_id = "BDV" (there are 844 such rows). For each site_id, we can compute the number of unique user_id's found in these 844 rows. Which site_id has the largest number of unique users? And what's the number?

In [24]:
q = """
SELECT country_id, site_id, count(DISTINCT user_id) as nunique_user
FROM db
WHERE country_id = "BDV"
GROUP BY site_id
ORDER BY 3 DESC
LIMIT 1
"""

runQuery(q)

,country_id,site_id,nunique_user
0,BDV,5NPAU,544


# Q2

Between 2019-02-03 00:00:00 and 2019-02-04 23:59:59, there are four users who visited a certain site more than 10 times. Find these four users & which sites they (each) visited more than 10 times. (Simply provides four triples in the form (user_id, site_id, number of visits) in the box below.)

In [35]:
q = """
SELECT ts, user_id, site_id, COUNT(DISTINCT ts) as nunique_visits
FROM db
WHERE ts BETWEEN '2019-02-03 00:00:00' AND '2019-02-04 23:59:59'
GROUP BY user_id, site_id
HAVING nunique_visits > 10
"""
runQuery(q)

,ts,user_id,site_id,nunique_visits
0,2019-02-03 19:48:09.000000,LC06C3,N0OTG,25
1,2019-02-03 02:05:39.000000,LC3A59,N0OTG,26
2,2019-02-03 00:02:31.000000,LC3C7E,3POLC,15
3,2019-02-04 06:11:19.000000,LC3C9D,N0OTG,17


# Q3

For each site, compute the unique number of users whose last visit (found in the original data set) was to that site. For instance, user "LC3561"'s last visit is to "N0OTG" based on timestamp data. Based on this measure, what are top three sites? (hint: site "3POLC" is ranked at 5th with 28 users whose last visit in the data set was to 3POLC; simply provide three pairs in the form (site_id, number of users).)

First, get the last 'ts' for each user.

In [136]:
q = """
SELECT DISTINCT user_id, MAX(ts) OVER (PARTITION BY user_id) as last_ts
FROM db
"""
runQuery(q)

,user_id,last_ts
0,LC00C3,2019-02-03 18:52:50.000000
1,LC01C3,2019-02-04 11:35:10.000000
2,LC05C3,2019-02-02 14:14:44.000000
3,LC06C3,2019-02-07 01:16:12.000000
4,LC07C3,2019-02-05 19:06:42.000000
...,...,...
1911,LCFC3B,2019-02-05 04:53:03.000000
1912,LCFC3D,2019-02-01 18:59:50.000000
1913,LCFC3E,2019-02-01 20:49:13.000000
1914,LCFEC3,2019-02-07 06:23:59.000000


Then, join it with the original db to get the last visited site_id for each user.

In [137]:
q = """
SELECT db.user_id, db.site_id as last_site
FROM db
JOIN
(SELECT DISTINCT user_id, MAX(ts) OVER (PARTITION BY user_id) as last_ts
FROM db) last
ON db.user_id = last.user_id AND db.ts = last.last_ts
"""
runQuery(q)

,user_id,last_site
0,LC39B6,N0OTG
1,LC3500,N0OTG
2,LCC1C3,QGO3G
3,LCC3C3,5NPAU
4,LC39C8,QGO3G
...,...,...
1911,LC3561,N0OTG
1912,LC3837,RT9Z6
1913,LC3F13,QGO3G
1914,LC35EB,QGO3G


Group by last_site from the above table and count the number of user_id.

In [153]:
q = """
SELECT db.site_id as site_id, COUNT(DISTINCT db.user_id) as n_user
FROM db
JOIN
(SELECT DISTINCT user_id, MAX(ts) OVER (PARTITION BY user_id) as last_ts
FROM db) last
ON db.user_id = last.user_id AND db.ts = last.last_ts
GROUP BY 1
ORDER BY 2 DESC
LIMIT 3
"""
runQuery(q)

,site_id,n_user
0,5NPAU,992
1,N0OTG,561
2,QGO3G,289


# Q4

For each user, determine the first site he/she visited and the last site he/she visited based on the timestamp data. Compute the number of users whose first/last visits are to the same website. What is the number?

## Include users with only 1 record

There are some users who have only 1 visit out of all sites. These users have automatically the same first and last site visits.

From Q3, we obtained the table of last visited sites for each user.

In [154]:
q = """
SELECT db.user_id, db.site_id as last_site
FROM db
JOIN
(SELECT DISTINCT user_id, MAX(ts) OVER (PARTITION BY user_id) as last_ts
FROM db) last
ON db.user_id = last.user_id AND db.ts = last.last_ts
"""
runQuery(q)

,user_id,last_site
0,LC39B6,N0OTG
1,LC3500,N0OTG
2,LCC1C3,QGO3G
3,LCC3C3,5NPAU
4,LC39C8,QGO3G
...,...,...
1911,LC3561,N0OTG
1912,LC3837,RT9Z6
1913,LC3F13,QGO3G
1914,LC35EB,QGO3G


Do the same to get the first visited site_id for each user.  
Then, join the first site table and last site table.  
Filter with records where the last visited site is the same as the first visited site.

In [139]:
q = """
SELECT iq.user_id, iq.last_site, iq.first_site
FROM(

(SELECT db.user_id, db.site_id as last_site, last.last_ts
FROM db
JOIN
(SELECT DISTINCT user_id, MAX(ts) OVER (PARTITION BY user_id) as last_ts
FROM db) last
ON db.user_id = last.user_id AND db.ts = last.last_ts) join1

JOIN

(SELECT db.user_id, db.site_id as first_site, first.first_ts
FROM db
JOIN
(SELECT DISTINCT user_id, MIN(ts) OVER (PARTITION BY user_id) as first_ts
FROM db) first
ON db.user_id = first.user_id AND db.ts = first.first_ts) join2

ON join1.user_id = join2.user_id

) iq
WHERE iq.last_site = iq.first_site

"""
runQuery(q)

,user_id,last_site,first_site
0,LC39B6,N0OTG,N0OTG
1,LC3500,N0OTG,N0OTG
2,LCC1C3,QGO3G,QGO3G
3,LCC3C3,5NPAU,5NPAU
4,LC39C8,QGO3G,QGO3G
...,...,...,...
1665,LCC36A,N0OTG,N0OTG
1666,LC34B8,QGO3G,QGO3G
1667,LC3F13,QGO3G,QGO3G
1668,LC35EB,QGO3G,QGO3G


Count the number of user_id's from the above table.

In [226]:
q = """
SELECT COUNT(*) as n_user
FROM(

(SELECT db.user_id, db.site_id as last_site, last.last_ts
FROM db
JOIN
(SELECT DISTINCT user_id, MAX(ts) OVER (PARTITION BY user_id) as last_ts
FROM db) last
ON db.user_id = last.user_id AND db.ts = last.last_ts) join1

JOIN

(SELECT db.user_id, db.site_id as first_site, first.first_ts
FROM db
JOIN
(SELECT DISTINCT user_id, MIN(ts) OVER (PARTITION BY user_id) as first_ts
FROM db) first
ON db.user_id = first.user_id AND db.ts = first.first_ts) join2

ON join1.user_id = join2.user_id

) iq
WHERE iq.last_site = iq.first_site

"""
runQuery(q)

,n_user
0,1670


## Ignore users with only 1 record

In [204]:
q = """
SELECT user_id
FROM db
GROUP BY user_id
HAVING COUNT(user_id) > 1
"""
runQuery(q)

,user_id
0,LC06C3
1,LC0C32
2,LC0C3E
3,LC0CC3
4,LC0FC3
...,...
650,LCFC34
651,LCFC36
652,LCFC3E
653,LCFEC3


Use the above table for filtering user_id's after getting each of the first sites and the last sites for each user.

In [224]:
q = """
SELECT join1.user_id, join1.last_site, join2.first_site
FROM
(SELECT DISTINCT iq.user_id, iq.site_id as last_site
    FROM(
        SELECT DISTINCT db.user_id, db.ts, db.site_id
        FROM db
        JOIN
            (SELECT DISTINCT user_id, site_id, MAX(ts) OVER (PARTITION BY user_id) as last_ts
             FROM db ) last
        ON db.user_id = last.user_id AND db.ts = last.last_ts
        WHERE db.user_id in
                (SELECT user_id
                FROM db
                GROUP BY user_id
                HAVING COUNT(user_id) > 1)
    )iq )join1

JOIN

(SELECT DISTINCT iq.user_id, iq.site_id as first_site
    FROM(
        SELECT DISTINCT db.user_id, db.ts, db.site_id
        FROM db
        JOIN
            (SELECT DISTINCT user_id, site_id, MIN(ts) OVER (PARTITION BY user_id) as first_ts
             FROM db ) first
        ON db.user_id = first.user_id AND db.ts = first.first_ts
        WHERE db.user_id in
                (SELECT user_id
                FROM db
                GROUP BY user_id
                HAVING COUNT(user_id) > 1)
    )iq )join2

ON join1.user_id = join2.user_id

WHERE join1.last_site = join2.first_site
"""
runQuery(q)

,user_id,last_site,first_site
0,LC3DAD,N0OTG,N0OTG
1,LC37EA,QGO3G,QGO3G
2,LC305A,3POLC,3POLC
3,LC3222,QGO3G,QGO3G
4,LC8C37,5NPAU,5NPAU
...,...,...,...
404,LC3D80,N0OTG,N0OTG
405,LCDC3A,N0OTG,N0OTG
406,LC3780,QGO3G,QGO3G
407,LC33F8,QGO3G,QGO3G


Count the number of users from the above table.

In [225]:
q = """
SELECT COUNT(*) as n_users
FROM
(SELECT DISTINCT iq.user_id, iq.site_id as last_site
    FROM(
        SELECT DISTINCT db.user_id, db.ts, db.site_id
        FROM db
        JOIN
            (SELECT DISTINCT user_id, site_id, MAX(ts) OVER (PARTITION BY user_id) as last_ts
             FROM db ) last
        ON db.user_id = last.user_id AND db.ts = last.last_ts
        WHERE db.user_id in
                (SELECT user_id
                FROM db
                GROUP BY user_id
                HAVING COUNT(user_id) > 1)
    )iq )join1

JOIN

(SELECT DISTINCT iq.user_id, iq.site_id as first_site
    FROM(
        SELECT DISTINCT db.user_id, db.ts, db.site_id
        FROM db
        JOIN
            (SELECT DISTINCT user_id, site_id, MIN(ts) OVER (PARTITION BY user_id) as first_ts
             FROM db ) first
        ON db.user_id = first.user_id AND db.ts = first.first_ts
        WHERE db.user_id in
                (SELECT user_id
                FROM db
                GROUP BY user_id
                HAVING COUNT(user_id) > 1)
    )iq )join2

ON join1.user_id = join2.user_id

WHERE join1.last_site = join2.first_site
"""
runQuery(q)

,n_users
0,409
